In [1]:
from mysql.connector import connect, Error
import pandas as pd
from faker import Faker
import random
from datetime import datetime
from calendar import monthrange

In [2]:
#initialize faker
fake = Faker()

In [3]:
categories = ["Groceries", "stationary", "shopping", "entertainment", "travel", "health", "bills", "rent", "Investments", "other"]
payment_modes = ["Cash", "Debit Card", "UPI", "Netbanking", "Credit Card"]

descriptions = {
    "Groceries": [
        "bought fruits from Mart",
        "bought vegetables from Mart",
        "bought meat from store",
        "bought some households from store"
    ],
    "stationary": [
        "bought pen from store",
        "bought notebook from store",
        "bought pencil from store",
        "bought scissors from store"
    ],
    "shopping": [
        "bought clothes from mall",
        "bought shoes from mall",
        "bought accessories from mall",
        "bought bags from mall"
    ],
    "entertainment": [
        "bought movie tickets",
        "bought concert tickets",
        "bought event tickets",
        "bought party tickets"
    ],
    "travel": [
        "bought flight tickets",
        "bought train tickets",
        "bought bus tickets",
        "bought taxi tickets"
    ],
    "health": [
        "bought medicines",
        "bought supplements",
        "health checkup expenses",
        "bought health insurance"
    ],
    "bills": [
        "paid electricity bill",
        "paid water bill",
        "paid internet bill",
        "paid phone bill"
    ],
    "rent": [
        "paid house rent"
    ],
    "Investments": [
        "bought stocks",
        "bought mutual funds",
        "bought gold",
        "bought silver"
    ],
    "other": [
        "bought gifts",
        "bought gadgets"
    ]
}

In [4]:
connection = connect(
    host="localhost",
    user="root",  
    password="Jashu@#1234",  
)
cursor = connection.cursor()

In [35]:
connection.reconnect()

In [5]:
# Create a new database and table for storing expenses
create_db_query = """
CREATE DATABASE IF NOT EXISTS ExpenseDB;
USE ExpenseDB;

CREATE TABLE IF NOT EXISTS Expenses (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Date DATE,
    Category VARCHAR(50),
    Payment_Mode VARCHAR(50),
    Description TEXT,
    Amount DECIMAL(10, 2),
    Cashback DECIMAL(6, 2)
);
"""

# Reconnect to the database if not connected
if not connection.is_connected():
    connection.reconnect()

# Execute the query
cursor.execute(create_db_query, multi=True)


<generator object CMySQLCursor._execute_iter at 0x000001B89E463880>

In [6]:
# Function to generate data for each month
def gen_exp_data_per_month():
    exp_data_per_month = {}
    for month in range(1, 13):
        month_str = f"2025-{month:02d}"
        data = []
        for _ in range(120):
            # Generate a date within the specific month
            days_in_month = monthrange(2025, month)[1]
            start_date = datetime(2025, month, 1)
            end_date = datetime(2025, month, days_in_month)
            date = fake.date_between_dates(date_start=start_date, date_end=end_date)

            # Generate other fields
            category = random.choice(categories)
            cashback = round(random.uniform(50.0, 600.0), 1)  # Cap cashback between 50 and 600
            expense = {
                "Date": date,
                "Category": category,
                "Payment_Mode": random.choice(payment_modes),
                "Description": random.choice(descriptions[category]),
                "Amount": random.randint(100, 10000),
                "Cashback": cashback,
            }
            data.append(expense)
        exp_data_per_month[month_str] = pd.DataFrame(data)
    return exp_data_per_month

In [7]:
# Generate data
exp_data_per_month = gen_exp_data_per_month()

In [8]:
# MySQL connection and data insertion
try:
    connection = connect(
        host="localhost",
        user="root",  
        password="Jashu@#1234",  
        database="ExpenseDB"
    )
    cursor = connection.cursor()
except Error as e:
    print(f"Error occurred: {e}")

In [9]:
insert_query = """
    INSERT INTO Expenses (Date, Category, Payment_Mode, Description, Amount, Cashback)
    VALUES (%s, %s, %s, %s, %s, %s)
"""

try:
    for month, df in exp_data_per_month.items():
        if not df.empty:  # Check if DataFrame is not empty
            print(f"Processing data for month: {month}")
            
            # Extracting and formatting data for insertion
            data_to_insert = df[['Date', 'Category', 'Payment_Mode', 'Description', 'Amount', 'Cashback']].to_records(index=False).tolist()
            
            # Insert data into the table
            cursor.executemany(insert_query, data_to_insert)
            
            # Commit changes to the database
            connection.commit()
            
            # Print success message for the specific month
            print(f"Data for {month[:4]}/{month[5:]} inserted successfully.")
        
except Error as e:
    print(f"Error occurred: {e}")

finally:
    # Ensure resources are released properly
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("Database connection closed.")


Processing data for month: 2025-01
Data for 2025/01 inserted successfully.
Processing data for month: 2025-02
Data for 2025/02 inserted successfully.
Processing data for month: 2025-03
Data for 2025/03 inserted successfully.
Processing data for month: 2025-04
Data for 2025/04 inserted successfully.
Processing data for month: 2025-05
Data for 2025/05 inserted successfully.
Processing data for month: 2025-06
Data for 2025/06 inserted successfully.
Processing data for month: 2025-07
Data for 2025/07 inserted successfully.
Processing data for month: 2025-08
Data for 2025/08 inserted successfully.
Processing data for month: 2025-09
Data for 2025/09 inserted successfully.
Processing data for month: 2025-10
Data for 2025/10 inserted successfully.
Processing data for month: 2025-11
Data for 2025/11 inserted successfully.
Processing data for month: 2025-12
Data for 2025/12 inserted successfully.
Database connection closed.


In [10]:
# Set Pandas display options to show wide tables in one line
pd.set_option('display.max_columns', None)   # Show all columns
pd.set_option('display.width', 1000)         # Increase display width
pd.set_option('display.colheader_justify', 'left')  # Center align column headers

In [11]:
# Connect to MySQL database
try:
    connection = connect(
        host="localhost",
        user="root",       # Replace with your username
        password="Jashu@#1234",   # Replace with your password
        database="ExpenseDB"        # Replace with your database name
    )
    if connection.is_connected():
        cursor = connection.cursor()
        print("Successfully connected to the database.")
except Error as e:
    print(f"Error occurred: {e}")

Successfully connected to the database.


In [12]:
# Query 1: Total amount spent in each category
cursor.execute("SELECT Category, SUM(Amount) AS Total_Spent FROM Expenses GROUP BY Category;")
result1 = cursor.fetchall()
df1 = pd.DataFrame(result1, columns=["Category", "Total Spent"])
print("1. Total Amount Spent in Each Category:")
print(df1)

1. Total Amount Spent in Each Category:
  Category       Total Spent
0         health  723915.00 
1    Investments  758978.00 
2      Groceries  727336.00 
3         travel  793672.00 
4     stationary  737562.00 
5           rent  725458.00 
6       shopping  660842.00 
7          bills  659424.00 
8  entertainment  695438.00 
9          other  717060.00 


In [13]:
# Query 2: Total amount spent using each payment mode
cursor.execute("SELECT Payment_Mode, SUM(Amount) AS Total_Spent FROM Expenses GROUP BY Payment_Mode;")
result2 = cursor.fetchall()
df2 = pd.DataFrame(result2, columns=["Payment Mode", "Total Spent"])
print("\n2. Total Amount Spent Using Each Payment Mode:")
print(df2)



2. Total Amount Spent Using Each Payment Mode:
  Payment Mode Total Spent
0         Cash  1610992.00
1   Debit Card  1422403.00
2          UPI  1256866.00
3   Netbanking  1438460.00
4  Credit Card  1470964.00


In [14]:
# Query 3: Total cashback received across all transactions
cursor.execute("SELECT SUM(Cashback) AS Total_Cashback FROM Expenses;")
result3 = cursor.fetchone()
print("\n3. Total Cashback Received Across All Transactions:")
print(f"Total Cashback: ₹{result3[0]:,.2f}")


3. Total Cashback Received Across All Transactions:
Total Cashback: ₹473,165.80


In [15]:
# Query 4: Top 5 most expensive categories
cursor.execute("SELECT Category, SUM(Amount) AS Total_Spent FROM Expenses GROUP BY Category ORDER BY Total_Spent DESC LIMIT 5;")
result4 = cursor.fetchall()
df4 = pd.DataFrame(result4, columns=["Category", "Total Spent"])
print("\n4. Top 5 Most Expensive Categories:")
print(df4)


4. Top 5 Most Expensive Categories:
  Category     Total Spent
0       travel  793672.00 
1  Investments  758978.00 
2   stationary  737562.00 
3    Groceries  727336.00 
4         rent  725458.00 


In [18]:
# Query 5: Spending on transportation by payment mode
cursor.execute("SELECT Payment_Mode, SUM(Amount) AS Total_Spent FROM Expenses WHERE Category = 'Travel' GROUP BY Payment_Mode;")
result5 = cursor.fetchall()
df5 = pd.DataFrame(result5, columns=["Payment Mode", "Total Spent"])
print("\n5. Spending on Transportation by Payment Mode:")
print(df5)


5. Spending on Transportation by Payment Mode:
  Payment Mode Total Spent
0   Netbanking  149621.00 
1   Debit Card  196269.00 
2         Cash  159507.00 
3          UPI  117582.00 
4  Credit Card  170693.00 


In [19]:
# Query 6: Transactions resulting in cashback
cursor.execute("SELECT * FROM Expenses WHERE Cashback > 0;")
result6 = cursor.fetchall()
df6 = pd.DataFrame(result6, columns=["ID", "Date", "Category", "Payment Mode", "Description", "Amount", "Cashback"])
print("\n6. Transactions Resulting in Cashback:")
print(df6)


6. Transactions Resulting in Cashback:
      ID   Date        Category     Payment Mode Description                   Amount   Cashback
0        1  2025-01-01       health         Cash       bought health insurance  3704.00  210.20 
1        2  2025-01-05  Investments   Debit Card                 bought silver  6915.00  198.70 
2        3  2025-01-13       health   Debit Card              bought medicines  3770.00  248.20 
3        4  2025-01-02    Groceries          UPI       bought fruits from Mart  5166.00  110.50 
4        5  2025-01-06       health   Debit Card       health checkup expenses  2514.00  531.80 
...    ...         ...          ...          ...                           ...      ...      ...
1435  1436  2025-12-12   stationary   Debit Card         bought pen from store  1684.00  459.80 
1436  1437  2025-12-18     shopping          UPI         bought bags from mall  8657.00  361.90 
1437  1438  2025-12-19        bills  Credit Card               paid water bill  5954.00

In [20]:
# Query 7: Total spending in each month
cursor.execute("SELECT MONTH(Date) AS Month, SUM(Amount) AS Total_Spent FROM Expenses GROUP BY Month ORDER BY Month;")
result7 = cursor.fetchall()
df7 = pd.DataFrame(result7, columns=["Month", "Total Spent"])
print("\n7. Total Spending in Each Month:")
print(df7)


7. Total Spending in Each Month:
    Month Total Spent
0    1     651364.00 
1    2     600502.00 
2    3     604754.00 
3    4     599856.00 
4    5     602511.00 
5    6     563759.00 
6    7     588354.00 
7    8     600215.00 
8    9     623293.00 
9   10     587914.00 
10  11     537774.00 
11  12     639389.00 


In [21]:
# Query 8: Months with highest spending in specific categories
cursor.execute("""
    SELECT MONTH(Date) AS Month, Category, SUM(Amount) AS Total_Spent 
    FROM Expenses 
    WHERE Category IN ('Travel', 'Entertainment', 'Gifts') 
    GROUP BY Month, Category ORDER BY Total_Spent DESC;
""")
result8 = cursor.fetchall()
df8 = pd.DataFrame(result8, columns=["Month", "Category", "Total Spent"])
print("\n8. Months with Highest Spending in Specific Categories:")
print(df8)


8. Months with Highest Spending in Specific Categories:
    Month Category       Total Spent
0    1            travel  111626.00 
1    8            travel  101447.00 
2    9            travel   84672.00 
3   10     entertainment   81698.00 
4    5     entertainment   73507.00 
5    7     entertainment   70146.00 
6   12            travel   65164.00 
7    6            travel   63975.00 
8   11            travel   63253.00 
9   11     entertainment   63040.00 
10   8     entertainment   62860.00 
11   5            travel   61415.00 
12  10            travel   58736.00 
13   2     entertainment   57460.00 
14  12     entertainment   54242.00 
15   4     entertainment   53627.00 
16   4            travel   53535.00 
17   7            travel   51627.00 
18   3            travel   51022.00 
19   3     entertainment   49070.00 
20   6     entertainment   45091.00 
21   9     entertainment   44537.00 
22   1     entertainment   40160.00 
23   2            travel   27200.00 


In [22]:
# Query 9: Average spending per transaction
cursor.execute("SELECT AVG(Amount) AS Avg_Spending FROM Expenses;")
result9 = cursor.fetchone()
print("\n9. Average Spending Per Transaction:")
print(f"Average Spending: ₹{result9[0]:,.2f}")


9. Average Spending Per Transaction:
Average Spending: ₹4,999.78


In [23]:
# Query 10: Most common spending day of the week
cursor.execute("""
    SELECT DAYNAME(Date) AS Day, COUNT(*) AS Transactions 
    FROM Expenses 
    GROUP BY Day 
    ORDER BY Transactions DESC LIMIT 1;
""")
result10 = cursor.fetchone()
print("\n10. Most Common Spending Day of the Week:")
print(f"Day: {result10[0]}, Transactions: {result10[1]}")


10. Most Common Spending Day of the Week:
Day: Wednesday, Transactions: 226


In [24]:
# Query 11: Spending trends by day of the week
cursor.execute("""
    SELECT DAYNAME(Date) AS Day, SUM(Amount) AS Total_Spent 
    FROM Expenses 
    GROUP BY Day 
    ORDER BY MIN(DAYOFWEEK(Date));
""")
result11 = cursor.fetchall()
df11 = pd.DataFrame(result11, columns=["Day", "Total Spent"])
print("\n11. Spending Trends by Day of the Week:")
print(df11)


11. Spending Trends by Day of the Week:
  Day        Total Spent
0     Sunday   893752.00
1     Monday  1106120.00
2    Tuesday   968245.00
3  Wednesday  1069842.00
4   Thursday   930380.00
5     Friday  1116199.00
6   Saturday  1115147.00


In [25]:
# Query 12: Categories with zero spending
cursor.execute("""
    SELECT Category 
    FROM (SELECT Category, SUM(Amount) AS Total_Spent FROM Expenses GROUP BY Category) AS subquery 
    WHERE Total_Spent = 0;
""")
result12 = cursor.fetchall()
df12 = pd.DataFrame(result12, columns=["Category"])
print("\n12. Categories with Zero Spending:")
print(df12)


12. Categories with Zero Spending:
Empty DataFrame
Columns: [Category]
Index: []


In [26]:
# Query 13: Percentage of cashback earned vs total spending
cursor.execute("SELECT SUM(Cashback) AS Total_Cashback, SUM(Amount) AS Total_Spent FROM Expenses;")
result13 = cursor.fetchone()
cashback_percentage = (result13[0] / result13[1]) * 100
print("\n13. Cashback Earned vs Total Spending:")
print(f"Cashback Percentage: {cashback_percentage:.2f}%")


13. Cashback Earned vs Total Spending:
Cashback Percentage: 6.57%


In [27]:
# Query 14: Highest spending in a single transaction
cursor.execute("SELECT * FROM Expenses ORDER BY Amount DESC LIMIT 1;")
result14 = cursor.fetchall()
df14 = pd.DataFrame(result14, columns=["ID", "Date", "Category", "Payment Mode", "Description", "Amount", "Cashback"])
print("\n14. Highest Spending in a Single Transaction:")
print(df14)       


14. Highest Spending in a Single Transaction:
   ID  Date        Category  Payment Mode Description               Amount   Cashback
0  822  2025-07-10  shopping  Cash         bought clothes from mall  9993.00  352.00 


In [28]:
# Query 15: Spending trends for recurring expenses
cursor.execute("""
    SELECT MONTH(Date) AS Month, Category, SUM(Amount) AS Total_Spent 
    FROM Expenses 
    WHERE Category IN ('Rent', 'Bills', 'Investments') 
    GROUP BY Month, Category 
    ORDER BY Month;
""")
result15 = cursor.fetchall()
df15 = pd.DataFrame(result15, columns=["Month", "Category", "Total Spent"])
print("\n15. Spending Trends for Recurring Expenses:")
print(df15)       


15. Spending Trends for Recurring Expenses:
    Month Category     Total Spent
0    1           bills  51830.00  
1    1     Investments  86085.00  
2    1            rent  69719.00  
3    2           bills  62306.00  
4    2     Investments  56974.00  
5    2            rent  92325.00  
6    3           bills   6714.00  
7    3     Investments  99262.00  
8    3            rent  62703.00  
9    4           bills  45024.00  
10   4     Investments  63448.00  
11   4            rent  83678.00  
12   5           bills  62353.00  
13   5     Investments  47626.00  
14   5            rent  47370.00  
15   6           bills  63648.00  
16   6     Investments  55661.00  
17   6            rent  94720.00  
18   7           bills  69175.00  
19   7     Investments  62578.00  
20   7            rent  50021.00  
21   8           bills  54738.00  
22   8     Investments  44196.00  
23   8            rent  43860.00  
24   9           bills  58997.00  
25   9     Investments  67587.00  
26   9    

In [29]:
# Query 16: Smallest transaction by amount
cursor.execute("SELECT * FROM Expenses ORDER BY Amount ASC LIMIT 1;")
result16 = cursor.fetchall()
df16 = pd.DataFrame(result16, columns=["ID", "Date", "Category", "Payment Mode", "Description", "Amount", "Cashback"])
print("\n16. Smallest Transaction by Amount:")
print(df16)


16. Smallest Transaction by Amount:
   ID Date        Category       Payment Mode Description           Amount  Cashback
0  87  2025-01-05  entertainment  Debit Card   bought movie tickets  100.00  89.80  


In [30]:
# Query 17: Spending distribution for weekend vs weekday
cursor.execute("""
    SELECT CASE 
        WHEN DAYOFWEEK(Date) IN (1, 7) THEN 'Weekend'
        ELSE 'Weekday'
    END AS Day_Type, SUM(Amount) AS Total_Spent 
    FROM Expenses 
    GROUP BY Day_Type;
""")
result17 = cursor.fetchall()
df17 = pd.DataFrame(result17, columns=["Day Type", "Total Spent"])
print("\n17. Spending Distribution for Weekend vs Weekday:")
print(df17)


17. Spending Distribution for Weekend vs Weekday:
  Day Type Total Spent
0  Weekday  5190786.00
1  Weekend  2008899.00


In [31]:
# Query 18: Highest cashback earned in a single transaction
cursor.execute("SELECT * FROM Expenses ORDER BY Cashback DESC LIMIT 1;")
result18 = cursor.fetchall()
df18 = pd.DataFrame(result18, columns=["ID", "Date", "Category", "Payment Mode", "Description", "Amount", "Cashback"])
print("\n18. Highest Cashback Earned in a Single Transaction:")
print(df18)



18. Highest Cashback Earned in a Single Transaction:
   ID  Date        Category Payment Mode Description     Amount   Cashback
0  508  2025-05-28  other    Credit Card  bought gadgets  5682.00  599.90 


In [32]:
try:
    # Your code here
    pass  # Replace this with the actual code that might raise an error

except Error as e:
    print(f"Error: {e}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("\nConnection closed.")


Connection closed.
